In [1]:
#!/usr/bin/env python3
import os
import argparse
import numpy as np
from tqdm import tqdm
import multiprocessing
import pickle
from joblib import Parallel, delayed
#import AMM_sim_functions as sim
import fast_simulation_with_arb as sim

# Input Parameters
eta0 = 0.000918 # CEX proportional cost
mu = 0.0  # Mean of CEX price shock
M = 20 # Total number of sims
thds = 7 # Get number of threads

# Fixed Market Parameters
T = 31 # Time horizon
N = 44638 # Number of periods
dt = T/N # Time increment

X = 23000000 # Initial CPMM Dollar Reserves
S = 3331.51 # Initial CEX Price
Y = X/S # Initial CPMM Asset Reserves
time = np.array([i * dt for i in range(N+1)]) # Time grid

# Generate buyer-first/seller-first trade filter
filtr_bfs , filtr_sfs =  sim.generate_trade_filters(N, M) 

# Set of AMM Fee values to test
eta1_vals = np.linspace(0.0001, 0.0035, 101)

# Set of sigma values for simulation
sigma_vals = np.linspace(0.02, 0.06, 21)

#buy_rates = np.linspace(8000., 50000., 20)
buy_rates = np.array([14704.])


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Collect data
all_outputs = {}  # Dictionary to store outputs keyed by sigma.

for k in range(len(buy_rates)):

    buy_rate = buy_rates[k] # Purchase rate per unit time for systematic buyers
    sell_rate = buy_rate # Sale rate per unit time for systematic sellers
    buy = buy_rate * dt # Constant trade size for systematic buyers
    sell = -sell_rate * dt # Constant trade size for systematic sellers

    for i in range(len(sigma_vals)):
        sigma_value = sigma_vals[i]
        print("Sigma Value: " + str(sigma_value))
        
        # Set the sigma for this iteration and simulate the CEX price series.
        sigma = sigma_value
        S0 = sim.CEX_Price(S, mu, sigma, dt, N, M)
        
        # Run the parallel simulation for each eta1 value.
        output_vals = Parallel(n_jobs=thds)(
            delayed(sim.fast_simulation_summary)(
                M, N, T, dt, buy, sell, eta0, eta1_vals[j], eta0, S0, X, Y, filtr_bfs, filtr_sfs
            )
            for j in tqdm(range(len(eta1_vals)))
        )
        
        # Store the output in the dictionary.
        all_outputs[sigma_value] = np.array(output_vals)

    # Save the dictionary to a file
    filename = "all_outputs_eta0_" + str(eta0) + "_mu_" + str(mu) \
                + "_buy_" + str(round(buy_rate)) + "_sell_" + str(round(sell_rate))+ ".pkl"

    with open(filename, "wb") as f:
        pickle.dump(all_outputs, f)

    print("Partial simulation done")

print("Simulation successfully completed.")

Sigma Value: 0.02


  7%|▋         | 7/101 [00:00<00:05, 18.28it/s]

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


100%|██████████| 101/101 [00:11<00:00,  9.01it/s]


Sigma Value: 0.022


...

Sigma Value: 0.057999999999999996


100%|██████████| 101/101 [00:11<00:00,  8.94it/s]


Sigma Value: 0.06


100%|██████████| 101/101 [00:08<00:00, 12.36it/s]


Partial simulation done
Simulation successfully completed.
